# 線形回帰

アニーリングを利用した線形回帰演算を説明します。

参考文献: https://arxiv.org/abs/2008.02355

### 線形回帰の損失関数

$n$ 個の変数を持つデータ(ベクトル)を $\boldsymbol{x}$ とし、それに対して予測したいターゲット値を $y$ とします。  
$y$ が各変数の重み付き和で予測可能と仮定すると、次のような重みベクトル $\boldsymbol{w}$ を考えることができます。

$$
\boldsymbol{x}^T \cdot \boldsymbol{w} = y
$$

次に、$m$ 個のデータからなる学習用データセット $X$ ($m \times n$ 行列)と ターゲットデータ $Y$ ($m$ 次元ベクトル)があるとします。  
$X$ に含まれる全てのデータ $\boldsymbol{x}$ について対応する $y$ を予測するような、1つの重みベクトル $\boldsymbol{w}$ を求めるのが線形回帰の目的です。

これは次のような関数の最小化問題となります。

$$
\min_{\boldsymbol{w}} E(\boldsymbol{w}) =  || X\boldsymbol{w}  - Y ||^2
$$

$E(\boldsymbol{w})$ を式変形すると

$$
\min_{\boldsymbol{w}} E(\boldsymbol{w}) = \boldsymbol{w}^T X^T X \boldsymbol{w} - 2\boldsymbol{w}^T X^T Y + Y^T Y
$$

ここで重み $\boldsymbol{w}$ を量子ビットの測定値にエンコードするために $\boldsymbol{w} = P\hat{\boldsymbol{w}}$ とします。  
$\hat{\boldsymbol{w}}$ は量子ビット測定値 $\{0, 1\}$ からなるベクトルです。  
また、最小化に影響しない $Y^T Y$ を省略します。

$$
\min_{\boldsymbol{w}} E(\boldsymbol{w}) = 　\hat{\boldsymbol{w}}^T P^T  X^T X P\hat{\boldsymbol{w}} - 2\hat{\boldsymbol{w}}^T P^T X^T Y
$$

すると次のように一般のQUBO形式の最小化問題に落とし込むことができるため、アニーリングで解くことができます。

$$
\min_{\boldsymbol{w}} E(\boldsymbol{w}) = 　\hat{\boldsymbol{w}}^T A \hat{\boldsymbol{w}} + \hat{\boldsymbol{w}}^T \boldsymbol{b} \\
A = P^T  X^T X P \\
\boldsymbol{b} = -2P^T X^T Y
$$

注目すべき点はQUBO行列のサイズが、データセットに含まれるデータの数に依存しない点です。  
(データの変数の数) $\times$ (重み値のエンコードに用いる量子ビット数) で決まります。  

これを blueqat を用いて実装しましょう。

In [73]:
from blueqat.wq import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

まずはデータセットを作成します。  
今回は求める重みをあらかじめ設定します。  
データセット $X$ をランダムに生成し、重みをかけてノイズを加えた値を計算しターゲットデータ $Y$ とします。

In [165]:
w = np.array([0.25, 0.75, 0.5])
X = np.random.rand(100, 3)

y = X @ w + np.random.normal(scale = 0.05, size = X.shape[0])

scikit-learn で古典的に線形回帰を行いましょう。  
ターゲットデータ $Y$ にノイズを加えているため若干のずれはありますが、あらかじめ設定した重みを精度良く推定できています。

In [166]:
from sklearn import linear_model

skmodel = linear_model.LinearRegression()
skmodel.fit(X, y)
w_sk = skmodel.coef_
print("Predicted weight:", w_sk)
print("True weight:", w)

Predicted weight: [0.23514126 0.72070147 0.48488868]
True weight: [0.25 0.75 0.5 ]


次にアニーリングで線形回帰を行います。  

アニーリングでは重み　$\boldsymbol{w}$ の各値をいくつかの量子ビットにエンコードします。  
どのような値にエンコードするかは任意に設定することができ、上の数式における変換行列 $P$ に対応します。  

ここでは簡単に、1つの重みパラメータを2量子ビットにエンコードし、$[0, 0.25, 0.5, 0.75]$ の4値から予測します。  
用いる量子ビット数は $(\text{エンコードに用いる量子ビット数}) \times (\text{変数の数}) = 2 \times 3 = 6$ です。

In [167]:
K = 2 # bit number for weight
d = 3 # Number of features

p = [2 ** (-i-1) for i in range(K)]
I = np.eye(d)
P = np.kron(I, p)

In [376]:
A = np.dot(np.dot(P.T, X.T), np.dot(X, P))
b = -2 * np.dot(np.dot(P.T, X.T), y)
QUBO = A + np.diag(b)
QUBO = np.triu(QUBO + QUBO.T) - np.eye(QUBO.shape[0]) * QUBO

In [377]:
a = Opt()
a.qubo = QUBO
res = a.run(shots = 10, sampler = 'fast')

In [378]:
res

[[0, 1, 1, 1, 1, 0],
 [1, 0, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 0],
 [1, 0, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 1, 0],
 [1, 0, 1, 1, 0, 1],
 [0, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 1, 0],
 [0, 1, 1, 1, 1, 0]]

アニーリングの結果を重みパラメータの値に変換し、元々設定していた値と比較します。

In [379]:
w_qa = P @ res[0]

print("Predicted weight:", w_qa)
print("True weight:", w)

Predicted weight: [0.25 0.75 0.5 ]
True weight: [0.25 0.75 0.5 ]


正しく予測できました。  
ここでは元々設定した重み $\boldsymbol{w}$ の値に合わせてエンコードを定めたため、正確に推定できています。  

実際は未知の重みを推定するため誤差が生じます。  
また誤差を小さく抑えるためにより多くの量子ビットを用意しエンコードの刻み幅を小さくする必要があります。